In [1]:
# SIMULATIONS NOTEBOOK

# This notebook runs a simulation for the Solar Dynamo model with the sABC algorithm.
# The results of the simulation, as well as the parameters used for the simulated data and 
# the simulation itself are all stored in files in the directory Simulations/Simulations i.
# The reason behind this is to have an easier access to the results of already run simulations,
# without running them again.

# NB: In this notebook, there's no function to visualize the results: the visualization is all
# contained in the "visualization.ipynb" notebook.

# RULES:

# There are two ways to use this notebook:
# 1) change all the parameters and then run all -> correct way
# 2) change things randomly and not in order and then run -> wrong way
# Please be careful, some functions change the directory in which everything is being saved; the
# order of the calling of the functions is organized to start from a generic dir, create the dir
# Simulations/Simulation i, go to that directory and then, when everythingis finished, go back to
# the initial dir!!! If you don't do that, it will stay in the subdir and at the next run it will
# create a subdir of a subdir -> if you need to stop midway through because u forgot something, 
# remember to come back to the initial directory (and eliminate the directory that has not correct
# files inside).

# GG EZ - kallo27

In [3]:
# NEEDED PACKAGES -> no visualization!!

using StochasticDelayDiffEq
using SpecialFunctions
using Distributions
using SimulatedAnnealingABC
using Distances
using DataFrames
using FFTW
using CSV

In [4]:
# FUNCTIONS NEDEED FOR THE MODEL

# Box-shaped function for the magnetic field range 
function f(B, B_max = 10, B_min = 1)
  return 1 / 4 * (1 .+ erf.(B .^ 2 .- B_min ^ 2)) .* (1 .- erf.(B .^ 2 .- B_max ^ 2))
end

# Model function for the DDE
function MagneticField(du, u, h, p, t)
  N, T, tau, sigma, Bmax = p
  q = T / tau

  B, dB = u

  du[1] = dB
  du[2] = - ((2 / tau) * dB + (B / tau^2) + N * h(p, t - q)[1] * f(h(p, t - q)[1], Bmax))
end

# Noise function for the DDE
function noise!(du, u, h, p, t)
  N, T, tau, sigma, Bmax = p
  du[1] = (sigma * Bmax)
end

# Distance function in the sABC algorithm
function f_dist(θ::Vector{Float64}; type::Int64 = 1, indeces::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120, fourier_data::Vector{Float64})
  prob = SDDEProblem(MagneticField, noise!, B0, h, tspan, θ)
  sol = solve(prob, EM(), dt = 0.01)
  
  simulated_data = sol[1,:]
  fourier_transform = fft(simulated_data)
  fourier_stats = abs.(fourier_transform[indeces])

  rho = [euclidean(fourier_stats[i], fourier_data[i]) for i in 1:length(fourier_stats)]
  return rho
end

# function for the summary statistics
function reduced_fourier_spectrum(u::Vector{Float64}, indeces::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120)
  fourier_transform = fft(u)
  return abs.(fourier_transform[indeces])
end

reduced_fourier_spectrum (generic function with 2 methods)

In [5]:
# FUNCTIONS NEEDED FOR SAVING THE RESULTS OF A SIMULATION

# function to create a new directory for each simulations, in order to store the needed files
function create_directory()
  base_path = pwd()
  base_path = joinpath(base_path, "Simulations")
  i = 1
  dir_name = "Simulation $i"
  dir_path = joinpath(base_path, dir_name)
  
  while isdir(dir_path)
    i += 1
    dir_name = "Simulation $i"
    dir_path = joinpath(base_path, dir_name)
  end
  
  mkpath(dir_path)
  println("Directory created at: $dir_path")
  cd(dir_path)
end

# function to save the params of the simulated data
function write_params_sim(par::Vector{Float64}, tspan::Tuple{Int, Int}, dt::Float64, B0::Vector{Float64}, h0::Vector{Float64}, noise0::Vector{Float64})
  df = DataFrame(
    Parameter = ["N", "T", "tau", "sigma", "B_max", "tspan", "dt", "B0", "h", "noise0"],
    Value = [par[1], par[2], par[3], par[4], par[5], tspan, dt, B0, h0, noise0]
  )

  CSV.write("parameters.csv", df)
end

# function to save the solution of the SDDE for the simulated data
function save_solution(sol::Union{RODESolution, Vector{RODESolution}})
  curr_path = pwd()
  filename = "simulated_sol.csv"
  path = joinpath(curr_path, filename)

  solution_df = DataFrame(Time = sol.t, u = sol[1, :], du = sol[2, :])
  CSV.write(filename, solution_df)
  println("Solution saved to file: $path")
end

# function to save the prior as a string
function get_prior_string(prior)
  parts = []
  for d in prior.dists
    if isa(d, Uniform)
      push!(parts, "Uniform($(minimum(d)), $(maximum(d)))")
    else
      error("Unsupported distribution type: $(typeof(d))")
    end
  end
  
  return "product_distribution(" * join(parts, ", ") * ")"
end

# function to save the sabc parameters
function save_sabc_params(prior, n_particles::Int, n_simulation::Int, v::Float64, type::Int, indeces::Union{Vector{Int}, StepRange{Int64, Int64}})
  curr_path = pwd()
  filename = "sabc_params.csv"
  path = joinpath(curr_path, filename)
    
  sabc_params = DataFrame(
    Parameter = ["prior", "n_particles", "n_simulation", "v", "type", "indeces"],
    Value = [get_prior_string(prior), n_particles, n_simulation, v, type, string(indeces)]
  )
    
 CSV.write(filename, sabc_params) 
 println("Parameters saved to: $path")
end

# Function to save the result object of a sABC algorithm
function save_result(result::SimulatedAnnealingABC.SABCresult{Vector{Float64}, Float64})
  curr_path = pwd()
  filenames = ["eps_hist.csv", "u_hist.csv", "rho_hist.csv"]
  variables = [result.state.ϵ_history, result.state.u_history, result.state.ρ_history]

  for (filename, variable) in zip(filenames, variables)
    labels = string.(1:size(variable, 1))
    path = joinpath(curr_path, filename)
    CSV.write(path, DataFrame(variable, labels))
    println("$filename data saved to: $path")
  end

  filename = "pop_hist.csv"
  path = joinpath(curr_path, filename)

  param_samples = hcat(result.population...)

  posterior_params = DataFrame(
    N_value = param_samples[1, :],
    T_value = param_samples[2, :],
    tau_value = param_samples[3, :],
    sigma_value = param_samples[4, :],
    Bmax_value = param_samples[5, :]
  )

  CSV.write(path, posterior_params)
  println("Posterior parameters saved to: $path")
end

save_result (generic function with 1 method)

In [6]:
# DIRECTORY MANAGING

# Current directory
initial_directory = pwd()

# New directory
create_directory()

# NB: After "create_directory", we move to the new directory.
# DON'T RUN THIS AGAIN, wait for the simulation to finish!!!! If you made errors,
# eliminate the Simulations/Simulation i directory and then rerun everything

Directory created at: /home/ubuntu/LCP_B/Project/Simulations/Simulation 3


In [14]:
# SIMULATED DATA PARAMETERS MANAGING

# Parameters for the simulated data
N = 6.2
T = 3.1
tau = 3.5
sigma = 0.04
B_max = 6.0

par = [4.969013671589867, 6.244768157731836, 0.543695331904234, 0.14564961346866517, 6.126734729146558]

# Time
tspan = (971, 1899)
dt = 0.01

# Initial conditions
B0 = [3.0, 0.0]
h0 = [0.0, 0.0]
noise0 = [1.0]

# Writing on file "parameters.csv" of the values set in this cell.
write_params_sim(par, tspan, dt, B0, h0, noise0)

"parameters.csv"

In [15]:
# DATA SIMULATION

# Data simulation with the chosen parameter
h(p, t) = h0
prob = SDDEProblem(MagneticField, noise!, B0, h, tspan, par)
sol = solve(prob, EM(), dt = dt)

# Important solution data that are necessary afterwards
t = sol.t
u = sol[1, :]
du = sol[2, :]

# Writing of the solution of the SDDE on file "simulated_sol.csv"
save_solution(sol)

Solution saved to file: /home/ubuntu/LCP_B/Project/Simulations/Simulation 3/simulated_sol.csv


In [39]:
# SIMULATION PARAMETERS MANAGING

# Parameters that can be tuned for new simulations
prior = product_distribution(Uniform(1, 15), Uniform(0.1, 15.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))
n_particles = 50
n_simulation = 1000
v = 1.0
type = 1
indeces = 1:6:120

# Writing on file "sabc_params.csv" of the values set in this cell.
save_sabc_params(prior, n_particles, n_simulation, v, type, indeces)

Parameters saved to: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1/sabc_params.csv


In [41]:
# SIMULATION

# Creation of the summary statistics from the simulated data
sim_ss = reduced_fourier_spectrum(u, indeces)

# Actual usage of the sABC algorithm
result = sabc(f_dist, prior;
              n_particles = n_particles, 
              n_simulation = n_simulation,
              v = v,
              type = type,
              indeces = indeces,
              fourier_data = sim_ss)

# Display of the summary of the results
display(result)

# Saving the results to the files: "eps_hist.csv", "u_hist.csv", "rho_hist.csv", "pop_hist.csv".
save_result(result)

[ Info: Using threads: 4 
[ Info: Set BLAS threads = 1 
[ Info: Set 'pinthreads(:cores)' for optimal multi-threading performance


[ Info: Initializing population...
[ Info: Initial resampling (δ = 0.1) - ESS = 37.76702414932687 
[ Info: Population with 50 particles initialised.
[ Info: Initial ϵ = [2.0341526303500066, 0.5886215697014994, 0.7185173673578837, 0.425227320268728, 0.4890305031772345, 8160.253218257152, 1.1236737011494347, 0.8164807714285626, 1.0047645086465602, 0.5325242126909748, 0.8029189739982614, 0.45226972265677473, 0.9516103461664837, 1.0255174529289028, 1.0308339160255031, 0.8449202708997415, 0.7409171919282967, 0.883172260146317, 0.9655695358071648, 0.6123449324274345]
[ Info: 2024-05-30T17:56:32.549 -- Starting population updates.
[ Info: 2024-05-30T17:56:52.478  All particles have been updated 19 times.


Approximate posterior sample with 50 particles:
  - simulations used: 1000
  - average transformed distance: 0.3303
  - ϵ: [0.3073, 0.2909, 0.2601, 0.3655, 0.371, 1.032, 0.5657, 0.8079, 0.3334, 0.4491, 0.3041, 0.3119, 0.3894, 0.5344, 0.7757, 0.9782, 0.7912, 0.4994, 0.4139, 0.3688]
  - population resampling: 1
  - acceptance rate: 0.08105
The sample can be accessed with the field `population`.
The history of ϵ can be accessed with the field `state.ϵ_history`.
 -------------------------------------- 


eps_hist.csv data saved to: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1/eps_hist.csv
u_hist.csv data saved to: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1/u_hist.csv
rho_hist.csv data saved to: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1/rho_hist.csv
Posterior parameters saved to: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1/population.csv


In [10]:
# DIRECTORY MANAGING

# WE go back to the initial directory
cd(initial_directory)
pwd()

"/home/ubuntu/LCP_B/Project"